# Análisis Exploratorio de Datos: Estudio del Dengue en Sonora utilizando SQLite3

In [157]:
# Importar las librarías SQLite3 y Pandas.
import sqlite3
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [158]:
# Nombre de tu archivo de base de datos
db_filename = './database/enfermedades_db.db'

In [159]:
# Obtén la ruta completa del archivo de la base de datos
db_path = os.path.join(os.getcwd(), db_filename)

In [160]:
# Crea una conección a la base de datos SQLite.
connection = sqlite3.connect(db_path)

In [161]:
# Crea un objeto Cursor para ejecutar instrucciones en SQL.
cursor = connection.cursor()

In [162]:
# Consulta SQL para obtener la lista de tablas
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
tables = cursor.fetchall()

# Imprimir la lista de tablas
print("Tablas en la base de datos:")
for table in tables:
    print(table[0])


Tablas en la base de datos:
municipio
dictamen
entidad
estatus_caso
institucion
resultado_pcr
sexo
si_no
tipo_paciente
registro


## 1. ¿Cuales son los municipios de sonora con más casos de dengue?


In [163]:
# Comenzamos investigando qué periodos de tiempo comprende la base de datos
query_periodo_inicial = """
SELECT *
FROM registro
ORDER BY FECHA_ACTUALIZACION ASC
LIMIT 1;
"""
# Lectura de datos desde SQLite
df_inicio = pd.read_sql_query(query_periodo_inicial, connection)

# Imprimir el DataFrame resultante
df_inicio

,Unnamed: 0,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,3861,2022-01-06,925362,1,44,5,28,2,2,5,...,2,2,2,2,5.0,1,5,3,5,28


In [164]:
# Comenzamos investigando qué periodos de tiempo comprende la base de datos
query_periodo_final = """
SELECT *
FROM registro
ORDER BY FECHA_ACTUALIZACION DESC
LIMIT 1;
"""
# Lectura de datos desde SQLite
df_final = pd.read_sql_query(query_periodo_final, connection)

# Imprimir el DataFrame resultante
df_final

,Unnamed: 0,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,56274,2022-12-12,941045,2,43,24,23,2,2,24,...,2,2,2,2,5.0,1,5,3,24,23


Podemos ver que la base de datos tiene registros desde el 06 de enero de 2022 hasta el 12 de diciembre del 2022. Procedemos entonces a investigar la pregunta central.

In [165]:
query_municipios = """
    SELECT
        MUNICIPIO_DESC,
        COUNT(*) AS TOTAL_CASOS
    FROM
        (
            SELECT
                municipio.MUNICIPIO AS MUNICIPIO_DESC,
                estatus_caso.DESCRIPCIÓN AS ESTATUS_DESC
            FROM
                registro
                INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave
                INNER JOIN estatus_caso ON registro.ESTATUS_CASO = estatus_caso.clave
                INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
            WHERE
                ENTIDAD_RES = 26
                AND ESTATUS_DESC = 'CONFIRMADO'
                AND DICTAMEN IN (1, 4, 5)
        ) AS filtered_data
    GROUP BY
        MUNICIPIO_DESC;
"""

# Lectura de datos desde SQLite
df_municipios = pd.read_sql_query(query_municipios, connection)

# Imprimir el DataFrame resultante
df_municipios

,MUNICIPIO_DESC,TOTAL_CASOS
0,AGUA PRIETA,4
1,ALAMOS,3544
2,ALTAR,6
3,ARIVECHI,9
4,BENITO JUÁREZ,256
5,BÁCUM,277
6,CABORCA,235
7,CAJEME,3812
8,CANANEA,18
9,EMPALME,176


In [166]:
# Crear la gráfica de barras horizontales interactiva con Plotly Express
df_municipios = df_municipios.sort_values(by='TOTAL_CASOS')
fig = px.bar(df_municipios, x='TOTAL_CASOS', y='MUNICIPIO_DESC',
             labels={'TOTAL_CASOS': 'Total de Casos', 'MUNICIPIO_DESC': 'Municipio'},
             title='Total de Casos de Dengue por Municipio')

# Mostrar la gráfica interactiva
fig.show()

Como podemos ver en la gráfica anterior Navojoa es el municipio con mayor cantidad de casos de dengue confirmados. Esto se pone interesante cuando contrastamos con el resto de municipios, ya que parece tener más del doble de casos. Sospechamos que esto puede deberse a las condiciones climáticas de Navojoa.

### 1.1 ¿Navojoa será el municipio con mayor casos de dengue confirmados historicamente?

In [167]:
# Crea la gráfica interactiva con Plotly Express
fig = px.line(df_municipios_hist, x='FECHA_ACTUALIZACION', y='TOTAL_CASOS', color='MUNICIPIO_DESC',
              labels={'TOTAL_CASOS': 'Total de Casos', 'FECHA_ACTUALIZACION': 'Fecha de Actualización'},
              title='Comportamiento Histórico de Total_Casos por Municipio')

# Muestra la gráfica interactiva
fig.show()

## 2. ¿Cómo se distribuye la cantidad de casos confirmados por dengue? 

Comenzamos creando un dataframe con la información de los registros que nos interesan, así generamos una tabla con las variables:

- ID_REGISTRO
- SEXO 
- EDAD_ANOS 
- MUNICIPIO_RES
- DICTAMEN
- RESULTADO_PCR
- ESTATUS_CASO
- dictamen.DESCRIPCIÓN
- estatus_caso.DESCRIPCIÓN 

donde el estado es Sonora, esto es, ENTIDAD_RES es igual a 26.

In [168]:
# Consulta SQL
query = """
    SELECT
        ID_REGISTRO,
        FECHA_ACTUALIZACION,
        EDAD_ANOS,
        RESULTADO_PCR,
        ESTATUS_CASO,
        dictamen.DESCRIPCIÓN AS DICTAMEN_DESC,
        estatus_caso.DESCRIPCIÓN AS ESTATUS_DESC,
        municipio.MUNICIPIO AS MUNICIPIO_DESC,
        sexo.DESCRIPCIÓN AS SEXO_DESC
    FROM
        registro
        INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave
        INNER JOIN estatus_caso ON registro.ESTATUS_CASO = estatus_caso.clave
        INNER JOIN sexo ON registro.SEXO = sexo.CLAVE
        INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
    WHERE
        ENTIDAD_RES = 26
        AND ESTATUS_DESC = 'CONFIRMADO'
        AND DICTAMEN IN (1, 4, 5);
"""

# Lectura de datos desde SQLite
df = pd.read_sql_query(query, connection)

# Imprimir el DataFrame resultante
df.head()

,ID_REGISTRO,FECHA_ACTUALIZACION,EDAD_ANOS,RESULTADO_PCR,ESTATUS_CASO,DICTAMEN_DESC,ESTATUS_DESC,MUNICIPIO_DESC,SEXO_DESC
0,910978,2022-01-06,39,2,2,NO APLICA,CONFIRMADO,BÁCUM,MUJER
1,911682,2022-01-06,12,5,2,NO APLICA,CONFIRMADO,BÁCUM,MUJER
2,926816,2022-01-06,29,5,2,NO APLICA,CONFIRMADO,NAVOJOA,MUJER
3,925132,2022-01-06,28,5,2,NO APLICA,CONFIRMADO,CAJEME,MUJER
4,923452,2022-01-06,13,3,2,NO APLICA,CONFIRMADO,CAJEME,HOMBRE


In [169]:
sexo = df["SEXO_DESC"].value_counts()

# Crear un DataFrame para el gráfico
df_total_sexo = sexo.reset_index()
df_total_sexo.columns = ['SEXO_DESC', 'Total_Casos']
df_total_sexo

,SEXO_DESC,Total_Casos
0,MUJER,12884
1,HOMBRE,10129


In [170]:
# Crear gráfico de barras
fig = px.bar(df_total_sexo, x='SEXO_DESC', y="Total_Casos", color='SEXO_DESC',
             title='Diferencia entre Hombres y Mujeres en Total de Casos',
             labels={'count': 'Total de Casos', 'SEXO_DESC': 'Género'},
             color_discrete_sequence=['purple', 'teal'])

# Mostrar el gráfico interactivo
fig.show()

In [171]:
# Crear histograma interactivo con Plotly
fig = px.histogram(df, x='EDAD_ANOS', color='SEXO_DESC', title='Histograma de distribución del Dengue por Sexo',
                   labels={'EDAD_ANOS': 'Edad (años)', 'count': 'Cantidad'},
                   opacity=0.8,  # Transparencia de las barras
                   color_discrete_sequence=['purple', 'teal']
                   )

# Mostrar el gráfico interactivo
fig.show()

In [172]:
df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23013 entries, 0 to 23012
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_REGISTRO          23013 non-null  int64         
 1   FECHA_ACTUALIZACION  23013 non-null  datetime64[ns]
 2   EDAD_ANOS            23013 non-null  int64         
 3   RESULTADO_PCR        23013 non-null  int64         
 4   ESTATUS_CASO         23013 non-null  int64         
 5   DICTAMEN_DESC        23013 non-null  object        
 6   ESTATUS_DESC         23013 non-null  object        
 7   MUNICIPIO_DESC       23013 non-null  object        
 8   SEXO_DESC            23013 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 1.6+ MB


### 

In [173]:
# Consulta SQL
query = """
    SELECT
        ID_REGISTRO,
        FECHA_ACTUALIZACION,
        EDAD_ANOS,
        RESULTADO_PCR,
        ESTATUS_CASO,
        dictamen.DESCRIPCIÓN AS DICTAMEN_DESC,
        estatus_caso.DESCRIPCIÓN AS ESTATUS_DESC,
        municipio.MUNICIPIO AS MUNICIPIO_DESC,
        sexo.DESCRIPCIÓN AS SEXO_DESC,
        resultado_pcr.DESCRIPCIÓN as PCR_DESC
    FROM
        registro
        INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave
        INNER JOIN estatus_caso ON registro.ESTATUS_CASO = estatus_caso.clave
        INNER JOIN sexo ON registro.SEXO = sexo.CLAVE
        INNER JOIN resultado_pcr ON registro.RESULTADO_PCR = resultado_pcr.CLAVE
        INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
    WHERE
        ENTIDAD_RES = 26
        AND ESTATUS_DESC = 'CONFIRMADO'
        AND DICTAMEN IN (1, 4, 5);
"""

# Lectura de datos desde SQLite
df = pd.read_sql_query(query, connection)

# Imprimir el DataFrame resultante
df

,ID_REGISTRO,FECHA_ACTUALIZACION,EDAD_ANOS,RESULTADO_PCR,ESTATUS_CASO,DICTAMEN_DESC,ESTATUS_DESC,MUNICIPIO_DESC,SEXO_DESC,PCR_DESC
0,910978,2022-01-06,39,2,2,NO APLICA,CONFIRMADO,BÁCUM,MUJER,DENV2
1,911682,2022-01-06,12,5,2,NO APLICA,CONFIRMADO,BÁCUM,MUJER,SIN SEROTIPO AISLADO
2,926816,2022-01-06,29,5,2,NO APLICA,CONFIRMADO,NAVOJOA,MUJER,SIN SEROTIPO AISLADO
3,925132,2022-01-06,28,5,2,NO APLICA,CONFIRMADO,CAJEME,MUJER,SIN SEROTIPO AISLADO
4,923452,2022-01-06,13,3,2,NO APLICA,CONFIRMADO,CAJEME,HOMBRE,DENV3
...,...,...,...,...,...,...,...,...,...,...
23008,985071,2022-12-12,72,5,2,NO APLICA,CONFIRMADO,CAJEME,HOMBRE,SIN SEROTIPO AISLADO
23009,985081,2022-12-12,55,5,2,NO APLICA,CONFIRMADO,NAVOJOA,MUJER,SIN SEROTIPO AISLADO
23010,985200,2022-12-12,24,2,2,NO APLICA,CONFIRMADO,HERMOSILLO,MUJER,DENV2
23011,985192,2022-12-12,85,2,2,EN ESTUDIO,CONFIRMADO,NAVOJOA,MUJER,DENV2


In [174]:
df_pcr = df['PCR_DESC'].value_counts()
# Crear un DataFrame para el gráfico
df_pcr = df_pcr.reset_index()
df_pcr.columns = ['PCR_DESC', 'Total_Casos']
df_pcr

,PCR_DESC,Total_Casos
0,SIN SEROTIPO AISLADO,16460
1,DENV2,6056
2,DENV3,463
3,DENV1,34


In [175]:
# Crear gráfico de barras
fig = px.bar(
    df_pcr,
    x='PCR_DESC',
    y='Total_Casos',
    color='PCR_DESC',
    title='Diferencia entre los diferentes serotipos de dengue confirmados en el estado',
    labels={'Total_Casos': 'Total de Casos', 'PCR_DESC': 'Serotipo'},
    color_discrete_sequence=['purple', 'teal', 'green', 'orange']  # Specify four distinct colors
)

# Mostrar el gráfico interactivo
fig.show()


In [ ]:
query_pcr_hist = """
    SELECT
        FECHA_ACTUALIZACION,
        PCR_DESC,
        COUNT(*) AS TOTAL_CASOS
    FROM
        (
            SELECT
                FECHA_ACTUALIZACION,
                resultado_pcr.DESCRIPCIÓN as PCR_DESC,
                estatus_caso.DESCRIPCIÓN AS ESTATUS_DESC,
                dictamen.DESCRIPCIÓN AS DICTAMEN_DESC
            FROM
                registro
                INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave
                INNER JOIN estatus_caso ON registro.ESTATUS_CASO = estatus_caso.clave
                INNER JOIN resultado_pcr ON registro.RESULTADO_PCR = resultado_pcr.clave
                INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
            WHERE
                ENTIDAD_RES = 26
                AND ESTATUS_DESC = 'CONFIRMADO'
                AND DICTAMEN IN (1, 4, 5)
        ) AS filtered_data
    GROUP BY
        FECHA_ACTUALIZACION, PCR_DESC;
"""

# Lectura de datos desde SQLite
df_pcr_hist = pd.read_sql_query(query_pcr_hist, connection)

# Imprimir el DataFrame resultante
df_pcr_hist

,FECHA_ACTUALIZACION,PCR_DESC,TOTAL_CASOS
0,2022-01-06,DENV2,26
1,2022-01-06,DENV3,1
2,2022-01-06,SIN SEROTIPO AISLADO,108
3,2022-01-24,SIN SEROTIPO AISLADO,1
4,2022-01-31,SIN SEROTIPO AISLADO,1
...,...,...,...
101,2022-12-05,SIN SEROTIPO AISLADO,1459
102,2022-12-12,DENV1,12
103,2022-12-12,DENV2,1308
104,2022-12-12,DENV3,132


In [ ]:
# Crea la gráfica interactiva con Plotly Express
fig = px.line(df_pcr_hist, x='FECHA_ACTUALIZACION', y='TOTAL_CASOS', color='PCR_DESC',
              labels={'TOTAL_CASOS': 'Total de Casos', 'FECHA_ACTUALIZACION': 'Fecha de Actualización'},
              title='Comportamiento Histórico de Total_Casos por tipo de serotipo')

# Muestra la gráfica interactiva
fig.show()

In [ ]:
connection.close()